In [10]:
import urllib.request, urllib.error, urllib.parse
import os, sys, re
import subprocess
import base64
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime as dt
import datetime
import pandas as pd

# common key
auth_key = "bcd151073c03b352e1ef2fd66c32209da9ca0afa"
key_offset = 0

def req_res(url, headers):
    """ do req. and get res.
    """
    req = urllib.request.Request(url, None, headers)
    res = urllib.request.urlopen(req)
    return req, res

def auth1():
    """ get 1st auth
    """
    url = "https://radiko.jp/v2/api/auth1"
    headers, auth_response = {}, {}
    headers = {
                        "User-Agent": "curl/7.56.1",
                        "Accept": "*/*",
                        "X-Radiko-App":"pc_html5",
                        "X-Radiko-App-Version":"0.0.1",
                        "X-Radiko-User":"dummy_user",
                        "X-Radiko-Device":"pc",
                    }
    req, res = req_res(url, headers)
    auth_response["body"], auth_response["headers"] = res.read(), res.info()
    return auth_response

def get_partial_key(auth_response):
    """ get a key and an authtoken
    """
    authtoken = auth_response["headers"]["x-radiko-authtoken"]
    offset, length   = int(auth_response["headers"]["x-radiko-keyoffset"]), int(auth_response["headers"]["x-radiko-keylength"])
    partialkey= auth_key[offset:offset + length]
    partialkey = base64.b64encode(partialkey.encode())
    return [partialkey,authtoken]

def auth2(partialkey, auth_token):
    """ get 2nd auth
    """
    url = "https://radiko.jp/v2/api/auth2"
    headers =  {
                        "X-Radiko-AuthToken": auth_token,
                        "X-Radiko-Partialkey": partialkey,
                        "X-Radiko-User": "dummy_user",
                        "X-Radiko-Device": 'pc'
                    }
    req, res = req_res(url, headers)
    txt = res.read()
    area = txt.decode()
    return area

def get_stations_ids(area):
    """ get station_name based on station_id
    """
    url = 'http://radiko.jp/v2/station/list/{}.xml'.format(area.split(',')[0])
    res = requests.get(url ,verify=False)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, "html.parser")
    station_id_lst = [i.text for i in soup.findAll('id')]
    station_name_lst = [i.text for i in soup.findAll('name')]
    return station_id_lst, station_name_lst

In [11]:
res = auth1()
ret = get_partial_key(res)
partialkey, token = ret[0], ret[1]
area = auth2(partialkey, token)
station_id_lst, station_name_lst = get_stations_ids(area)

pd.DataFrame({'station_name':station_name_lst
                         ,'station_id':station_id_lst})

,station_name,station_id
0,TBSラジオ,TBS
1,文化放送,QRR
2,ニッポン放送,LFR
3,ラジオNIKKEI第1,RN1
4,ラジオNIKKEI第2,RN2
5,InterFM897,INT
6,TOKYO FM,FMT
7,J-WAVE,FMJ
8,ラジオ日本,JORF
9,bayfm78,BAYFM78
